# MultiSceneDataset 使用示例

本 notebook 演示如何使用 `MultiSceneDataset` 进行多场景数据加载，支持 EVolSplat 的 feed-forward 3DGS 训练。

## 主要功能
1. 多场景管理（训练/评估场景分离）
2. 基于关键帧的场景分段
3. 段内随机选择 source/target 关键帧
4. 打包成 EVolSplat 格式的批次数据
5. **后台线程预加载**：类似 torch DataLoader 的 worker 线程，持续预加载场景，确保训练队列满
6. **线程安全**：所有队列和缓存操作都使用锁保护，支持多线程场景加载
7. **阻塞等待机制**：场景切换时，如果场景未加载完成，主线程阻塞等待，确保数据就绪


In [ ]:
import os
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
from omegaconf import OmegaConf
from pathlib import Path

# 添加项目路径
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from datasets.multi_scene_dataset import MultiSceneDataset

# 设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


## 1. 配置数据集


In [ ]:
# 配置数据路径
data_cfg = OmegaConf.create({
    'data_root': '/path/to/your/data',  # 修改为你的数据路径
    'dataset': 'nuscenes',  # 或 'waymo', 'kitti' 等
    'start_timestep': 0,
    'end_timestep': -1,
    'preload_device': 'cpu',
    'pixel_source': {
        'type': 'datasets.nuscenes.nuscenes_sourceloader.NuScenesPixelSource',
        'cameras': [0, 1, 2],
        'downscale_when_loading': [3, 3, 3],
        'downscale': 1,
        'undistort': False,
        'test_image_stride': 0,
        'load_sky_mask': True,
        'load_dynamic_mask': True,
        'load_depth_maps': True,
        'load_objects': True,
        'load_smpl': False,
    },
    'lidar_source': {},
})

# 创建数据集实例
dataset = MultiSceneDataset(
    data_cfg=data_cfg,
    train_scene_ids=[0, 1, 2, 3, 4],  # 训练场景ID列表
    eval_scene_ids=[5, 6],  # 评估场景ID列表
    num_source_keyframes=3,  # 每个批次使用的源关键帧数量
    num_target_keyframes=6,  # 每个批次使用的目标关键帧数量
    segment_overlap_ratio=0.2,  # 段之间的重叠比例
    keyframe_split_config={
        'num_splits': 0,  # 0 表示自动确定分割数量
        'min_count': 1,
        'min_length': 0.0,
    },
    min_keyframes_per_scene=10,  # 每个场景最少需要的关键帧数
    min_keyframes_per_segment=6,  # 每个段最少需要的关键帧数
    device=device,
    preload_scene_count=3,  # 预加载的场景数量
    fixed_segment_aabb=None,  # 可选：固定段的AABB边界框
)

print(f"数据集已创建，训练场景数: {len(dataset.train_scene_ids)}, 评估场景数: {len(dataset.eval_scene_ids)}")


## 2. 初始化数据集


In [ ]:
# 初始化数据集（可选，会在第一次使用时自动初始化）
dataset.initialize()

# 获取当前场景ID
current_scene_id = dataset.get_current_scene_id()
print(f"当前训练场景: {current_scene_id}")


## 3. 获取批次数据


In [ ]:
# 方式1: 获取随机批次
batch = dataset.sample_random_batch()

print(f"批次信息:")
print(f"  场景ID: {batch['scene_id'].item()}")
print(f"  段ID: {batch['segment_id']}")
print(f"  源关键帧数量: {len(batch['source_keyframes'])}")
print(f"  目标关键帧数量: {len(batch['target_keyframes'])}")


In [ ]:
# 方式2: 获取指定场景和段的批次
batch = dataset.get_segment_batch(scene_id=0, segment_id=2)

print(f"指定场景和段的批次:")
print(f"  场景ID: {batch['scene_id'].item()}")
print(f"  段ID: {batch['segment_id']}")


## 4. 获取场景信息


In [ ]:
# 获取场景信息
scene_info = dataset.get_scene(scene_id=0)
if scene_info:
    print(f"场景 0 信息:")
    print(f"  段数量: {len(scene_info['segments'])}")
    print(f"  总帧数: {scene_info['num_frames']}")
    print(f"  相机数量: {scene_info['num_cams']}")
    
    # 查看第一个段的信息
    if len(scene_info['segments']) > 0:
        segment = scene_info['segments'][0]
        print(f"\n第一个段信息:")
        print(f"  关键帧范围: {segment.get('keyframe_range', 'N/A')}")
        print(f"  AABB: {segment.get('aabb', 'N/A')}")


## 5. 在训练循环中使用


In [ ]:
# 方式1: 使用 sample_random_batch（简单方式）
for iteration in range(10):  # 示例：10次迭代
    batch = dataset.sample_random_batch()
    scene_id = batch['scene_id'].item()
    segment_id = batch['segment_id']
    
    # 使用批次进行训练
    # loss = model(batch)
    # loss.backward()
    # optimizer.step()
    
    if iteration % 5 == 0:
        print(f"迭代 {iteration}: 场景 {scene_id}, 段 {segment_id}")
    
    # 如果场景训练完成，标记并切换到下一个场景
    # dataset.mark_scene_completed(scene_id)


In [ ]:
# 方式2: 使用调度器（推荐方式，更灵活）
scheduler = dataset.create_scheduler(
    batches_per_segment=20,  # 每个段生成20个批次
    segment_order="random",  # 段顺序："random" 或 "sequential"
    scene_order="random",  # 场景顺序："random" 或 "sequential"
    shuffle_segments=True,  # 是否打乱段
    preload_next_scene=True,  # 是否预加载下一个场景
)

try:
    for iteration, batch in enumerate(scheduler):
        scene_id = batch['scene_id'].item()
        segment_id = batch['segment_id']
        
        # 使用批次进行训练
        # loss = model(batch)
        # loss.backward()
        # optimizer.step()
        
        if iteration % 10 == 0:
            print(f"迭代 {iteration}: 场景 {scene_id}, 段 {segment_id}")
        
        # 可以在这里添加训练逻辑
        
except StopIteration:
    print("所有批次已处理完成")


## 6. 获取段的关键帧


In [ ]:
# 获取指定场景和段的关键帧列表
frame_indices = dataset.get_segment_frames(scene_id=0, segment_id=0)
print(f"场景 0, 段 0 的关键帧: {frame_indices[:10]}..." if len(frame_indices) > 10 else f"场景 0, 段 0 的关键帧: {frame_indices}")


## 7. 获取单帧数据


In [ ]:
# 获取指定场景、帧和相机的数据
frame_data = dataset.get_frame_data(
    scene_id=0,
    frame_idx=10,
    cam_idx=0,
)

if frame_data:
    print(f"帧数据键: {list(frame_data.keys())}")
    if 'image' in frame_data:
        print(f"图像形状: {frame_data['image'].shape}")
    if 'depth' in frame_data:
        print(f"深度图形状: {frame_data['depth'].shape}")


## 8. 可视化（可选）


In [ ]:
# 示例：可视化源图像和目标图像
batch = dataset.sample_random_batch()

# 获取第一个源关键帧的图像
if len(batch['source_keyframes']) > 0:
    source_frame = batch['source_keyframes'][0]
    source_data = dataset.get_frame_data(
        scene_id=batch['scene_id'].item(),
        frame_idx=source_frame,
        cam_idx=0,
    )
    
    if source_data and 'image' in source_data:
        plt.figure(figsize=(10, 5))
        plt.subplot(1, 2, 1)
        plt.imshow(source_data['image'])
        plt.title(f"源关键帧 {source_frame}")
        plt.axis('off')
        
        # 获取第一个目标关键帧的图像
        if len(batch['target_keyframes']) > 0:
            target_frame = batch['target_keyframes'][0]
            target_data = dataset.get_frame_data(
                scene_id=batch['scene_id'].item(),
                frame_idx=target_frame,
                cam_idx=0,
            )
            
            if target_data and 'image' in target_data:
                plt.subplot(1, 2, 2)
                plt.imshow(target_data['image'])
                plt.title(f"目标关键帧 {target_frame}")
                plt.axis('off')
                
                plt.tight_layout()
                plt.show()


## 注意事项

1. **数据路径配置**：确保 `data_cfg.data_root` 指向正确的数据目录
2. **场景ID**：确保 `train_scene_ids` 和 `eval_scene_ids` 中的场景ID在数据集中存在
3. **内存管理**：`preload_scene_count` 控制预加载的场景数量，根据可用内存调整
4. **线程安全**：数据集支持多线程场景加载，但确保在使用时遵循线程安全的最佳实践
5. **批次格式**：返回的批次数据格式符合 EVolSplat 训练要求
